In [13]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from pprint import pprint
%matplotlib inline
import scipy.stats as sps



In [19]:
dataset = sb.load_dataset( "titanic")
dataset['target']=dataset['survived']

In [23]:
def entropy(target_col):
    elements, counts = np.unique(target_col,return_counts = True)
    entropy = np.sum( [(-counts[i]/np.sum(counts))* np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    return entropy

def InfoGain(data,split_attribute_name,target_name = 'class'):
    
    total_entropy = entropy(data[target_name])
    
    vals,counts = np.unique(data[split_attribute_name], return_counts = True)
    
    Weighted_entropy = np.sum([(counts[i] / np.sum(counts))*entropy(data.where(data[split_attribute_name]== vals[i]).dropna()[target_name]) for i in range(len(vals))])
    
    Information_gain = total_entropy - Weighted_entropy
    
    return Information_gain

def ID3(data,originaldata, features, target_attribute_name = 'class', parent_node_class = None ):
    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]
    
    elif len(data) == 0:
        return np.unique( originaldata[target_attribute_name]) [np.argmax(np.unique(originaldata[taget_attribute_name]
                                                                                     
                                    , return_counts = True )[1])]
    
    elif len(features) == 0:
        return parent_node_class
    
    else:
        parent_node_class = np.unique(data[target_attribute_name]) [np.argmax(np.unique(data[target_attribute_name]
                                                                                       
                                            ,return_counts = True)[1])]
        
        features = np.random.choice(features, size = np.int(np.sqrt(len(features))), replace = True)
        
        item_values = [InfoGain(data,feature, target_attribute_name) for feature in features ]
        
        best_feature_index = np.argmax(item_values)
        
        best_feature = features[ best_feature_index]
        
        tree = {best_feature : {}}
        
        features = [i for i in features if i != best_feature ]
        
        for value in np.unique(data[best_feature]):
            
            sub_data=data.where(data[best_feature] == value ).dropna()
            
            subtree = ID3(sub_data, dataset, features, target_attribute_name, parent_node_class )
            
            tree[best_feature][value] = subtree
            
        return(tree)
    
def predict(query, tree, default =  'p'):
    
    for key in list(query.key()) :
        if key in list(tree.keys()):
            
            try:
                result = tree[key][query[key]]
                
            except:
                return default
            
            if isinstance(result, dict):
                return predict(query, result)
            
            else:
                
                return result
            
def train_test_split(datset):
    
    training_data = dataset.iloc[: round(0.75 * len(dataset))].reset_index(drop = True)
    
    testing_data = dataset.iloc[round(0.75 * len(dataset)):].reset_index (drop = True)
    
    return training_data, testing_data 

training_data = train_test_split(dataset)[0]
testing_data = train_test_split(dataset)[0]



def RandomForest_Train(dataset, nu_of_trees):
    
    random_forest_sub_tree =[]
    
    for i in range( nu_of_trees):
        
        bootstrap_sample =  dataset.sample(frac =1, replace = True)
        
        bootstrap_training_data = train_test_split(bootstrap_sample)[0]
        
        bootstrap_testing_data = train_test_split(bootstrap_sample)[0]
        
        random_forest_sub_tree.append(ID3(bootstrap_training_data,bootstrap_training_data,bootstrap_training_data.drop(labels=['target'], axis=1).columns))
        
        return random_forest_sub_tree
    
random_forest = RandomForest_Train(dataset,50)



def RandomForest_ppredict (query, random_forest, default = 'p'):
    
    prediction=[]
    
    for tree in random_forest:
        
        prediction.append(predict(query,tree,default))
        
    return sps.mode (prediction)[0][0]


query = testing_data.iloc [0,:].drop("target").to_dict()

query_target = testing_data.iloc [0,0]

print('target', query_target)

prediction = RandomForest_Predict( query, random_forest)

print(" Prediction", prediction)


def RandomForest_Test (data, random_forest):
    
    data['prediction'] = None
    
    for i in range( len(data)):
        
        query =data.iloc [i,:].drop('target').to_dict()
        
        data.loc[i, 'prediction'] = RandomForest_predict( query, random_forest, default = 'p')
        
    accuracy = sum( data['prediction'] == data['target']) / len(data) * 100
    
    return accuracy

RandomForest_Test(testing_data, random_forest)



TypeError: '<' not supported between instances of 'float' and 'str'